In [1]:
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt


In [2]:
train_initial=pd.read_csv('train.csv')
test_initial=pd.read_csv('test.csv')
train_initial.head()

,id,person_age,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,0,37,35000,RENT,0.0,EDUCATION,B,6000,11.49,0.17,N,14,0
1,1,22,56000,OWN,6.0,MEDICAL,C,4000,13.35,0.07,N,2,0
2,2,29,28800,OWN,8.0,PERSONAL,A,6000,8.90,0.21,N,10,0
3,3,30,70000,RENT,14.0,VENTURE,B,12000,11.11,0.17,N,5,0
4,4,22,60000,RENT,2.0,MEDICAL,A,6000,6.92,0.10,N,3,0


In [3]:
sample=pd.read_csv('sample_submission.csv')
print(f"sample shape is {sample.shape} testing shape is {test_initial.shape}, training shape is {train_initial.shape}")

sample shape is (39098, 2) testing shape is (39098, 12), training shape is (58645, 13)


In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
#Preprocessing
from sklearn.preprocessing import LabelEncoder
label_encoder = LabelEncoder()
test_id=test_initial['id']
columns_to_drop=['id','person_age']
train_initial.drop(columns_to_drop,axis=1,inplace=True)
test_initial.drop(columns_to_drop,axis=1,inplace=True)
non_numeric_coulumns= [col for col in train_initial.columns if not pd.api.types.is_numeric_dtype(train_initial[col])]
for col in non_numeric_coulumns:
    train_initial[col]=label_encoder.fit_transform(train_initial[col])
    test_initial[col]=label_encoder.fit_transform(test_initial[col])
train_initial.head()

,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,35000,3,0.0,1,1,6000,11.49,0.17,0,14,0
1,56000,2,6.0,3,2,4000,13.35,0.07,0,2,0
2,28800,2,8.0,4,0,6000,8.90,0.21,0,10,0
3,70000,3,14.0,5,1,12000,11.11,0.17,0,5,0
4,60000,3,2.0,3,0,6000,6.92,0.10,0,3,0


In [6]:
from sklearn.preprocessing import StandardScaler

# Initialize the scaler
scaler = StandardScaler()

# List all columns except the target
numerical_columns = train_initial.columns.drop('loan_status')

# Fit and transform the scaler on the feature columns
train_initial[numerical_columns] = scaler.fit_transform(train_initial[numerical_columns])
test_initial[numerical_columns] = scaler.transform(test_initial[numerical_columns])
train_initial.head()

,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length,loan_status
0,-0.765768,0.913186,-1.187200,-0.881912,-0.063697,-0.578306,0.267616,0.117378,-0.417419,2.031798,0
1,-0.212128,0.224728,0.328047,0.278934,0.892168,-0.937775,0.880532,-0.973242,-0.417419,-0.946489,0
2,-0.929223,0.224728,0.833130,0.859357,-1.019563,-0.578306,-0.585854,0.553626,-0.417419,1.039036,0
3,0.156966,0.913186,2.348377,1.439780,-0.063697,0.500101,0.142396,0.117378,-0.417419,-0.201917,0
4,-0.106673,0.913186,-0.682117,0.278934,-1.019563,-0.578306,-1.238314,-0.646056,-0.417419,-0.698298,0


In [7]:
from sklearn.model_selection import train_test_split
X=train_initial.drop('loan_status',axis=1)
y=train_initial['loan_status']
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)

In [8]:
pip install tensorflow

Note: you may need to restart the kernel to use updated packages.


In [9]:
# Creating the model
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

from sklearn.metrics import roc_auc_score

callback = keras.callbacks.EarlyStopping(monitor='loss', patience=3)

model=keras.Sequential()
model.add(Dense(64,activation='relu', input_dim=X_train.shape[1]))
model.add(Dense(32,activation='relu'))
model.add(Dense(32,activation='relu'))
model.add(Dense(1,activation='sigmoid'))
model.compile(optimizer='adam',loss='binary_crossentropy',metrics=['accuracy'])

history = model.fit(X_train, y_train, epochs=20, validation_data=(X_test, y_test), batch_size=32, callbacks=[callback])
y_pred = model.predict(X_test)

# Calculate ROC-AUC score
roc_auc = roc_auc_score(y_test, y_pred)
print(f'ROC-AUC Score: {roc_auc:.4f}')

Epoch 1/20
1467/1467 [==============================] - 4s 2ms/step - loss: 0.2525 - accuracy: 0.9064 - val_loss: 0.2240 - val_accuracy: 0.9219
Epoch 2/20
1467/1467 [==============================] - 3s 2ms/step - loss: 0.2189 - accuracy: 0.9232 - val_loss: 0.2151 - val_accuracy: 0.9216
Epoch 3/20
1467/1467 [==============================] - 2s 2ms/step - loss: 0.2103 - accuracy: 0.9265 - val_loss: 0.2043 - val_accuracy: 0.9298
Epoch 4/20
1467/1467 [==============================] - 3s 2ms/step - loss: 0.2056 - accuracy: 0.9291 - val_loss: 0.2009 - val_accuracy: 0.9330
Epoch 5/20
1467/1467 [==============================] - 3s 2ms/step - loss: 0.2015 - accuracy: 0.9316 - val_loss: 0.2025 - val_accuracy: 0.9334
Epoch 6/20
1467/1467 [==============================] - 3s 2ms/step - loss: 0.1985 - accuracy: 0.9339 - val_loss: 0.1979 - val_accuracy: 0.9326
Epoch 7/20
1467/1467 [==============================] - 3s 2ms/step - loss: 0.1945 - accuracy: 0.9366 - val_loss: 0.1962 - val_accuracy:

In [10]:
test_initial.head()

,person_income,person_home_ownership,person_emp_length,loan_intent,loan_grade,loan_amnt,loan_int_rate,loan_percent_income,cb_person_default_on_file,cb_person_cred_hist_length
0,0.130602,0.913186,-0.429576,-0.301489,3.759765,2.836651,1.674687,2.189555,-0.417419,-0.946489
1,0.842425,-1.152188,0.328047,0.859357,0.892168,0.140632,0.659750,-0.646056,2.395674,-0.450108
2,-0.897587,0.913186,0.075506,1.439780,2.803899,-0.937775,2.145908,-0.318870,2.395674,-0.946489
3,-0.370311,0.913186,-0.177035,-1.462336,-1.019563,-0.398572,-0.585854,-0.209808,-0.417419,0.294464
4,1.000608,-1.152188,0.833130,-0.301489,1.848034,1.039305,1.859221,-0.100746,2.395674,-0.450108


In [11]:

y_pred_final=model.predict(test_initial)


1222/1222 [==============================] - 1s 1ms/step


In [12]:
y_pred_final=y_pred_final.flatten()


In [13]:
def create_df(data=[] , col_names=[]):

    sub_df = pd.DataFrame(data).T
    sub_df.columns = col_names
    return sub_df

In [14]:
test_id = test_id.tolist()

In [15]:
output=create_df([test_id , y_pred_final] , ['id' , 'loan_status'])
output['id'] = output['id'].convert_dtypes('int32')
output

,id,loan_status
0,58645,0.990736
1,58646,0.023377
2,58647,0.162855
3,58648,0.042072
4,58649,0.103734
...,...,...
39093,97738,0.030488
39094,97739,0.007387
39095,97740,0.024203
39096,97741,0.326267


In [16]:
output.to_csv('submission.csv',index=False)
